# Testing `numpy` inference

### Imports

In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [ ]:
import sys
import time
import pickle

#import numpy as np
import autograd.numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt


module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
%load_ext autoreload
%autoreload 2
np.set_printoptions(precision=3, linewidth=500, edgeitems=15, suppress=True)

import seaborn as sns
sns.set_palette('bright')


import matplotlib as mpl
import matplotlib.style as mplstyle

label_size = 9 
mpl.rcParams['xtick.labelsize'] = label_size 
mpl.rcParams['ytick.labelsize'] = label_size 

mplstyle.use('fast')

#from matplotlib import rc
#rc('text', usetex=False)

import utils.file_io as io

In [ ]:
from plotters.plot_observables import grid_plot, get_obs_dict
import utils.file_io as io

import pandas as pd
from plotters.plot_utils import load_pkl

from plotters.plot_observables import get_run_dirs, get_title_str
from lattice.lattice import u1_plaq_exact
import scipy
import datetime
import matplotlib as mpl
label_size = 9
mpl.rcParams['xtick.labelsize'] = label_size 
mpl.rcParams['ytick.labelsize'] = label_size 

In [ ]:
from params.gauge_params import GAUGE_PARAMS
from trainers.train_setup import train_setup
from utils.attr_dict import AttrDict
from models.gauge_model import GaugeModel

## Create `GaugeModel`

In [ ]:
from trainers.train_setup import train_setup
from params.gauge_params import GAUGE_PARAMS

_params = GAUGE_PARAMS.copy()
params, hooks = train_setup(_params, log_file=None)
params['zero_masks'] = True
params['space_size'] = 4
params['time_size'] = 4
params['batch_size'] = 8
for key, val in params.items():
    print(f'{key}: {val}')

In [ ]:
params['train_steps'] = 100

In [ ]:
from models.gauge_model import GaugeModel

model = GaugeModel(params)

In [ ]:
from loggers.train_logger import TrainLogger
from trainers.train_setup import create_config

train_logger = TrainLogger(model, model.log_dir,
                           logging_steps=model.logging_steps,
                           summaries=params['summaries'])
config, params = create_config(params)

sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())

In [ ]:
from trainers.trainer import Trainer
trainer = Trainer(sess, model, train_logger, **params)

In [ ]:
from config import NetWeights, NP_FLOAT

samples_init = np.array(model.lattice.samples_array, dtype=NP_FLOAT)

net_weights_init = NetWeights(1, 1, 1, 1, 1, 1)

trainer.train(model.train_steps,
              beta=model.beta_init,
              samples=samples_init,
              net_weights=net_weights_init)

In [ ]:
def pkl_dump(obj, out_file, name=''):
    io.log(f'Saving {name} to {out_file}')
    with open(out_file, 'wb') as f:
        pickle.dump(obj, f)

In [ ]:
from trainers.train_setup import get_net_weights

wfile = os.path.join(model.log_dir, 'dynamics_weights.h5')
model.dynamics.save_weights(wfile)

weights_final, coeffs_final = get_net_weights(model, sess)
xcoeffs = sess.run(list(coeffs_final['xnet'].values()))
vcoeffs = sess.run(list(coeffs_final['vnet'].values()))
weights_final['xnet']['GenericNet'].update({
    'coeff_scale': xcoeffs[0],
    'coeff_transformation': xcoeffs[1],
})
weights_final['vnet']['GenericNet'].update({
    'coeff_scale': vcoeffs[0],
    'coeff_transformation': vcoeffs[1],
})
pkl_dump(weights_final, os.path.join(model.log_dir, 'weights.pkl'), name='weights_final')
#pkl_dump(model.params, os.path.join(os.getcwd(), 'params.pkl'), name='model.params')
#io.save_dict(model.params, os.path.join(os.getcwd()), 'params')


 * Create `dynamics_np` to compare against `model.dynamics`:

In [ ]:
from runners.runner_np import create_dynamics, load_pkl, run_inference_np

eps_np = sess.run(model.dynamics.eps)
dynamics_np, lattice = create_dynamics(model.log_dir, eps=eps_np,
                                       num_steps=model.num_steps,
                                       batch_size=model.batch_size)

Create operations for calculating quantities of interest:   
 - `xf`: Proposed $x$, obtained from running `model.dyanmics.transition_kernel` with `forward=True`   
 - `vf`: Proposed $v$, obtained from running `model.dynamics.transition_kernel` with `forward=True`
 - `pxf`: $A(\xi^{\prime}|\xi)$
 - `sumlogdetf`: Log determinant, accumulated over all leapfrog steps. Given by:
 
 \begin{equation}
 \log|\mathcal{J}| = \log\left|\frac{\partial\xi^{\prime}}{\partial \xi^{T}}\right| = d \sum_{t\leq N_{\mathrm{LF}}} \left[\frac{\varepsilon}{2}\mathbb{1} \cdot S_{v}(\zeta^{t}_{1}) + \varepsilon m^{t} \cdot S_{x}(\zeta^{t}_{2}) + \varepsilon\bar{m}^{t} \cdot S_{x}(\zeta^{t}_{3}) + \frac{\varepsilon}{2}\mathbb{1}\cdot S_{v}(\zeta^{t}_{4})\right]
 \label{eq:sumlogdet}
 \end{equation}   
 
- `xf_`: $x^{\prime\prime}$, obtained by updating $x$ for a single leapfrog step.
- `vf_`: $v^{\prime\prime}$, obtained by updating $v$ for a single leapfrog step.
- `sld_`: Accumulated log determinant after a single leapfrog step. ($t = 1$ in Eq.\ref{eq:sumlogdet})
- `dudx_tf_`: $\partial_{x}U(x, \beta)$, evaluated at $\beta = 5$

In [ ]:
from config import State, TF_FLOAT
from seed_dict import seeds

# ------------------------------
# Run `model.dynamics` forward:
# ------------------------------
#vf_init = tf.random_normal(tf.shape(model.x), dtype=TF_FLOAT, seed=seeds['vf_init'], name='vf_init')
vf_init_np = np.array(np.random.randn(*model.x.shape), dtype=NP_FLOAT)
vf_init = tf.constant(vf_init_np)

state_init_f = State(model.x, vf_init, model.beta)
outf = model.dynamics.transition_kernel(*state_init_f,
                                        model.net_weights,
                                        model.train_phase,
                                        forward=True, hmc=True)
xf = outf['x_proposed']
vf = outf['v_proposed']
pxf = outf['accept_prob']
pxf_hmc = outf['accept_prob_hmc']
sumlogdetf = outf['sumlogdet']

In [ ]:
step = 0
# Create operations for getting the output from `model.dynamics._forward_lf`:
xf_, vf_, sld_, _ = model.dynamics._forward_lf(model.x, vf_init,
                                               model.beta, step,
                                               model.net_weights,
                                               training=model.train_phase)

# Create operation for calculating the gradient of the potential
dudx_tf_ = model.dynamics.grad_potential(model.x, model.beta)

In [ ]:
train_phase = False
t = model.dynamics._get_time(step, tile=tf.shape(model.x)[0])
Sv, Tv, Qv = model.dynamics.vnet([model.x, dudx_tf_, t], model.train_phase)

mask, mask_inv = model.dynamics._get_mask(step)
Sx, Tx, Qx = model.dynamics.xnet([vf_init, mask * model.x, t], model.train_phase)

 - Run `model.dynamics` forward and compare against results from running `dynamics_np` forward:

In [ ]:
step = 0
beta_np = 5.
train_phase = False
net_weights = NetWeights(1, 1, 1, 1, 1, 1)
#xf_init_np = np.zeros(model.x.shape, dtype=NP_FLOAT)
xf_init_np = np.array(np.random.randn(*model.x.shape), dtype=NP_FLOAT)
keys = ['xf', 'vf', 'vf_init', 'pxf', 'sumlogdetf']
fops = [xf, vf, vf_init, pxf, sumlogdetf]
_keys = ['xf_', 'vf_', 'sld_']
_fops = [xf_, vf_, sld_]


def forward_lf_tf(net_weights):
    feed_dict = {
        model.x: xf_init_np,
        model.beta: beta_np,
        model.net_weights: net_weights,
        model.train_phase: train_phase
    }
    
    _fout = sess.run(_fops, feed_dict=feed_dict)
    _fout_dict = dict(zip(_keys, _fout))
    
    return _fout_dict

    
def forward_lf_np(net_weights, vf_init_np):
    
    _fout_np = dynamics_np._forward_lf(xf_init_np, vf_init_np,
                                       beta_np, step, net_weights)
    _fout_dict_np = dict(zip(_keys, _fout_np))
    return _fout_dict_np


def dynamics_forward_tf(net_weights):
    feed_dict = {
        model.x: xf_init_np,
        model.beta: beta_np,
        model.net_weights: net_weights,
        model.train_phase: train_phase
    }

    fout = sess.run(fops, feed_dict=feed_dict)
    fout_dict = dict(zip(keys, fout))
    
    return fout_dict


def run_xnet():
    feed_dict = {
        model.x: xf_init_np,
        model.beta: beta_np,
        model.net_weights: net_weights,
        model.train_phase: train_phase,
    }
    
    outputs = sess.run([Sx, Tx, Qx], feed_dict=feed_dict)
    
    return outputs


def run_vnet():
    feed_dict = {
        model.x: xf_init_np,
        model.beta: beta_np,
        model.train_phase: train_phase,
    }
    outputs = sess.run([Sv, Tv, Qv], feed_dict=feed_dict)
    
    return outputs

def run_xnet_np():
    t = dynamics_np._get_time(step, tile=xf_init_np.shape[0])
    mask, mask_inv = dynamics_np._get_mask(step)
    outputs = dynamics_np.xnet([vf_init_np, mask * xf_init_np, t])
    return outputs


def run_vnet_np():
    t = dynamics_np._get_time(step, tile=xf_init_np.shape[0])
    mask, mask_inv = dynamics_np._get_mask(step)
    dU_dx = dynamics_np.grad_potential(xf_init_np, beta_np)
    outputs = dynamics_np.vnet([xf_init_np, dU_dx, t])
    return outputs
    

def dynamics_forward_np(net_weights, vf_init_np):
    state_init_f_np = State(xf_init_np, vf_init_np, beta_np)
    
    xf, vf, pxf, sldf = dynamics_np.transition_kernel(*state_init_f_np,
                                                      net_weights, forward=True)
    fout_dict_np = {
        'xf': xf,
        'vf': vf,
        'vf_init': vf_init_np,
        'pxf': pxf,
        'sumlogdetf': sldf,
    }
    
    return fout_dict_np


def calc_diff(x1, x2, name=''):
    diff = np.sqrt(np.sum((x1 - x2)**2))
    print(f'{name} diff = {diff}')
    
    return diff

 - Check that $\partial_x U(x)$ is the same for `dynamics_tf` and `dynamics_np`:

In [ ]:
xrand_np = np.array(np.random.randn(*xf_init_np.shape), dtype=NP_FLOAT)
dudx_np = dynamics_np.grad_potential(xrand_np, beta_np)

dudx_tf = sess.run(dudx_tf_, feed_dict={model.x: xrand_np, model.beta: beta_np})

diff = np.sqrt(np.sum((dudx_np - dudx_tf) ** 2))
print(f'Gradients agree: {diff < 1e-3}')

In [ ]:
Sx_tf, Tx_tf, Qx_tf = run_xnet()
Sx_np, Tx_np, Qx_np = run_xnet_np()
print(f'Sx agrees: {np.allclose(Sx_tf, Sx_np)}')
print(f'Tx agrees: {np.allclose(Tx_tf, Tx_np)}')
print(f'Qx agrees: {np.allclose(Qx_tf, Qx_np)}')

Sv_tf, Tv_tf, Qv_tf = run_vnet()
Sv_np, Tv_np, Qv_np = run_vnet_np()
print(f'Sv agrees: {np.allclose(Sv_tf, Sv_np)}')
print(f'Tv agrees: {np.allclose(Tv_tf, Tv_np)}')
print(f'Qv agrees: {np.allclose(Qv_tf, Qv_np)}')

In [ ]:
HEADER = 80 * '-'
net_weights_arr = [
    NetWeights(0, 0, 0, 0, 0, 0),
    #NetWeights(0, 0, 0, 0, 0, 1),
    #NetWeights(0, 0, 0, 0, 1, 0),
    #NetWeights(0, 0, 0, 1, 0, 0),
    #NetWeights(0, 0, 1, 0, 0, 0),
    #NetWeights(0, 1, 0, 0, 0, 0),
    #NetWeights(1, 0, 0, 0, 0, 0),
    NetWeights(1, 1, 1, 1, 1, 1),
]

diffs_dict = {
    
}
for net_weights in net_weights_arr:
    io.log(HEADER)
    io.log(f'NetWeights: {net_weights}')
    fout_tf = dynamics_forward_tf(net_weights)
    diffs_dict[net_weights] = {}
    _vf_np = fout_tf['vf_init']
    np.allclose(_vf_np, vf_init_np)
    fout_np = dynamics_forward_np(net_weights, vf_init_np)
    for key in fout_tf.keys():
        diffs_dict[key] = calc_diff(fout_tf[key], fout_np[key], name=key)
    io.log(HEADER)

In [ ]:
HEADER1 = (len(f'NetWeights: {tuple(net_weights)}') + 5) * '-'

_diffs_dict = {}
for net_weights in net_weights_arr:
    io.log(HEADER1)
    io.log(f'NetWeights: {tuple(net_weights)}')
    _diffs_dict[net_weights] = {}
    _fout_tf = forward_lf_tf(net_weights)
    #_vf_init = _fout_tf
    _fout_np = forward_lf_np(net_weights, vf_init_np)
    for key in _fout_tf.keys():
        _diffs_dict[key] = calc_diff(_fout_tf[key], _fout_np[key], name=key)